# Portfolio Monitoring Dashboard

**Purpose**: Weekly review of live AQR Multi-Factor ETF portfolio  
**Frequency**: Run every Monday morning  
**Runtime**: ~2-3 minutes  

---

## What This Notebook Does

This dashboard provides a comprehensive view of your current portfolio status:

1. **Position Status**: Current holdings vs target weights, drift analysis
2. **Factor Analysis**: Evolution of factor scores for current positions
3. **Risk Monitoring**: Stop-loss distances with VIX-adjusted thresholds
4. **Performance Tracking**: Returns since last rebalance and YTD
5. **Transaction Costs**: Year-to-date trading costs
6. **Rebalancing Alerts**: Triggers when drift exceeds 5% threshold

---

## Quick Start

**Just run all cells** (Kernel → Restart & Run All)

The notebook automatically loads the most recent data and generates all visualizations.

In [ ]:
# Standard imports
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Scientific computing
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Imports successful")
print(f"✓ Project root: {project_root}")
print(f"✓ Current date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

---

## 1. Load Current Portfolio Data

Load the most recent target portfolio and executed positions.

In [ ]:
# Define data paths
data_dir = project_root / 'data'
results_dir = project_root / 'results'
portfolio_dir = results_dir / 'live_portfolio'

# Find most recent target portfolio
target_files = sorted(portfolio_dir.glob('target_portfolio_*.csv'))
if not target_files:
    raise FileNotFoundError("No target portfolio found. Run scripts/07_run_live_portfolio.py first.")

latest_target_file = target_files[-1]
print(f"Loading target portfolio: {latest_target_file.name}")

# Load target weights
target_weights = pd.read_csv(latest_target_file, index_col=0)
target_weights.columns = ['weight']
target_weights = target_weights[target_weights['weight'] > 0].sort_values('weight', ascending=False)

print(f"\n✓ Target portfolio: {len(target_weights)} positions")
print(f"  Total weight: {target_weights['weight'].sum():.1%}")

# Check for executed portfolio (may not exist on first run)
executed_file = portfolio_dir / 'executed_portfolio.csv'
if executed_file.exists():
    executed_weights = pd.read_csv(executed_file, index_col=0)
    executed_weights.columns = ['weight']
    print(f"✓ Executed portfolio: {len(executed_weights[executed_weights['weight'] > 0])} positions")
else:
    executed_weights = pd.DataFrame(index=target_weights.index, columns=['weight'])
    executed_weights['weight'] = 0.0
    print("⚠️  No executed portfolio found - assuming new portfolio (all positions at 0%)")

# Display target portfolio
print("\n📊 Target Portfolio Weights:")
display(target_weights.style.format({'weight': '{:.2%}'}).background_gradient(cmap='Greens', subset=['weight']))

---

## 2. Load Price Data & Calculate Current Positions

Load the latest price data to determine current market values.

In [ ]:
# Load filtered ETF prices
prices_file = data_dir / 'processed' / 'etf_prices_filtered.parquet'
if not prices_file.exists():
    raise FileNotFoundError(f"Filtered prices not found: {prices_file}")

prices = pd.read_parquet(prices_file)
print(f"✓ Loaded {len(prices.columns)} ETF price series")
print(f"  Date range: {prices.index.min().date()} to {prices.index.max().date()}")
print(f"  Latest date: {prices.index.max().date()}")

# Get latest prices for portfolio holdings
latest_prices = prices[target_weights.index].iloc[-1]
print(f"\n✓ Latest prices for {len(latest_prices)} holdings:")
print(latest_prices.to_frame('price').style.format({'price': '${:.2f}'}))

# Calculate position drift if executed portfolio exists
if executed_file.exists():
    drift = target_weights['weight'] - executed_weights.reindex(target_weights.index, fill_value=0)['weight']
    drift_df = pd.DataFrame({
        'target': target_weights['weight'],
        'current': executed_weights.reindex(target_weights.index, fill_value=0)['weight'],
        'drift': drift,
        'drift_abs': drift.abs()
    })
    
    max_drift = drift_df['drift_abs'].max()
    total_drift = drift_df['drift_abs'].sum() / 2  # Divide by 2 to avoid double counting
    
    print(f"\n📏 Portfolio Drift Analysis:")
    print(f"  Maximum drift: {max_drift:.2%}")
    print(f"  Total drift: {total_drift:.2%}")
    print(f"  Rebalance threshold: 5.00%")
    
    if max_drift > 0.05:
        print("\n🚨 ALERT: Rebalancing recommended (drift > 5%)")
    else:
        print("\n✅ No rebalancing needed (drift < 5%)")
else:
    drift_df = None
    print("\n📝 New portfolio - no drift calculation")

---

## 3. Position Status Visualization

Compare target vs current weights with drift highlighting.

In [ ]:
if drift_df is not None:
    # Create comparison chart
    fig = go.Figure()
    
    # Current weights
    fig.add_trace(go.Bar(
        name='Current Weight',
        x=drift_df.index,
        y=drift_df['current'] * 100,
        marker_color='lightblue'
    ))
    
    # Target weights
    fig.add_trace(go.Bar(
        name='Target Weight',
        x=drift_df.index,
        y=drift_df['target'] * 100,
        marker_color='darkgreen'
    ))
    
    fig.update_layout(
        title='Portfolio Weights: Current vs Target',
        xaxis_title='ETF Ticker',
        yaxis_title='Weight (%)',
        barmode='group',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()
    
    # Drift visualization
    fig2 = go.Figure()
    
    colors = ['red' if abs(d) > 0.05 else 'orange' if abs(d) > 0.03 else 'green' 
              for d in drift_df['drift']]
    
    fig2.add_trace(go.Bar(
        x=drift_df.index,
        y=drift_df['drift'] * 100,
        marker_color=colors,
        name='Drift'
    ))
    
    # Add threshold lines
    fig2.add_hline(y=5, line_dash="dash", line_color="red", 
                   annotation_text="Rebalance Threshold (+5%)")
    fig2.add_hline(y=-5, line_dash="dash", line_color="red", 
                   annotation_text="Rebalance Threshold (-5%)")
    
    fig2.update_layout(
        title='Position Drift from Target',
        xaxis_title='ETF Ticker',
        yaxis_title='Drift (%)',
        height=500,
        hovermode='x'
    )
    
    fig2.show()
    
    # Display drift table
    print("\n📊 Detailed Drift Analysis:")
    display(drift_df[['target', 'current', 'drift']].style
            .format({'target': '{:.2%}', 'current': '{:.2%}', 'drift': '{:.2%}'})
            .background_gradient(cmap='RdYlGn_r', subset=['drift'], vmin=-0.1, vmax=0.1))
else:
    # Just show target weights for new portfolio
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=target_weights.index,
        y=target_weights['weight'] * 100,
        marker_color='darkgreen',
        name='Target Weight'
    ))
    
    fig.update_layout(
        title='Target Portfolio Weights (New Portfolio)',
        xaxis_title='ETF Ticker',
        yaxis_title='Weight (%)',
        height=500
    )
    
    fig.show()

---

## 4. Factor Score Analysis

Analyze the factor characteristics of current portfolio holdings.

In [ ]:
# Load factor scores
signals_dir = data_dir / 'signals'
factor_files = {
    'momentum': signals_dir / 'momentum_scores.parquet',
    'quality': signals_dir / 'quality_scores.parquet',
    'value': signals_dir / 'value_scores.parquet',
    'volatility': signals_dir / 'volatility_scores.parquet'
}

# Check if factor files exist
factors_available = all(f.exists() for f in factor_files.values())

if factors_available:
    # Load all factor scores
    factor_scores = {}
    for name, filepath in factor_files.items():
        factor_scores[name] = pd.read_parquet(filepath)
    
    print("✓ Loaded factor scores:")
    for name, df in factor_scores.items():
        print(f"  {name.capitalize()}: {df.shape}")
    
    # Get latest factor scores for portfolio holdings
    latest_factor_scores = pd.DataFrame({
        'momentum': factor_scores['momentum'][target_weights.index].iloc[-1],
        'quality': factor_scores['quality'][target_weights.index].iloc[-1],
        'value': factor_scores['value'][target_weights.index].iloc[-1],
        'volatility': factor_scores['volatility'][target_weights.index].iloc[-1]
    })
    
    # Add composite score (geometric mean)
    latest_factor_scores['composite'] = latest_factor_scores.prod(axis=1) ** (1/4)
    
    # Add weights for weighted analysis
    latest_factor_scores['weight'] = target_weights['weight']
    
    print("\n📊 Latest Factor Scores for Portfolio Holdings:")
    display(latest_factor_scores.sort_values('composite', ascending=False)
            .style.format({col: '{:.3f}' for col in latest_factor_scores.columns if col != 'weight'})
            .format({'weight': '{:.2%}'})
            .background_gradient(cmap='RdYlGn', subset=['momentum', 'quality', 'value', 'volatility', 'composite']))
    
    # Heatmap of factor scores
    fig = px.imshow(
        latest_factor_scores[['momentum', 'quality', 'value', 'volatility']].T,
        labels=dict(x="ETF", y="Factor", color="Score"),
        x=latest_factor_scores.index,
        y=['Momentum', 'Quality', 'Value', 'Volatility'],
        color_continuous_scale='RdYlGn',
        aspect='auto',
        title='Factor Scores Heatmap (Current Holdings)'
    )
    fig.update_layout(height=400)
    fig.show()
    
    # Portfolio-weighted average factor exposure
    weighted_factors = (latest_factor_scores[['momentum', 'quality', 'value', 'volatility']].T * 
                       latest_factor_scores['weight']).T.sum()
    
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        x=['Momentum', 'Quality', 'Value', 'Volatility'],
        y=weighted_factors.values,
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
        text=weighted_factors.round(3),
        textposition='auto'
    ))
    
    fig2.update_layout(
        title='Portfolio-Weighted Average Factor Exposure',
        yaxis_title='Weighted Average Score',
        height=400
    )
    fig2.show()
    
else:
    print("⚠️  Factor score files not found.")
    print("   Run the signal generation pipeline first to enable factor analysis.")

---

## 5. Risk Monitoring: Stop-Loss Analysis

Monitor how close each position is to its stop-loss threshold with VIX-adjusted levels.

In [ ]:
# Load VIX data if available
vix_file = data_dir / 'raw' / 'prices' / '^VIX.csv'
if vix_file.exists():
    vix_df = pd.read_csv(vix_file, index_col=0, parse_dates=True)
    vix_series = vix_df['Adj Close'].dropna()
    current_vix = vix_series.iloc[-1]
    
    # Determine VIX regime and stop-loss threshold
    if current_vix < 15:
        stop_loss_threshold = 0.15
        vix_regime = "Low Volatility"
    elif current_vix <= 25:
        stop_loss_threshold = 0.12
        vix_regime = "Normal Volatility"
    else:
        stop_loss_threshold = 0.10
        vix_regime = "High Volatility"
    
    print(f"📊 VIX Analysis:")
    print(f"  Current VIX: {current_vix:.2f}")
    print(f"  Regime: {vix_regime}")
    print(f"  Stop-Loss Threshold: {stop_loss_threshold:.0%}")
    
    # Calculate drawdowns from recent peaks for each position
    lookback_days = 60
    recent_prices = prices[target_weights.index].iloc[-lookback_days:]
    
    # Calculate running max and drawdown
    running_max = recent_prices.expanding().max()
    drawdowns = (recent_prices - running_max) / running_max
    current_drawdowns = drawdowns.iloc[-1]
    
    # Calculate distance to stop-loss
    distance_to_stop = stop_loss_threshold + current_drawdowns  # Positive means safe, negative means triggered
    
    # Create stop-loss monitoring dataframe
    stop_loss_df = pd.DataFrame({
        'current_drawdown': current_drawdowns,
        'stop_loss_threshold': -stop_loss_threshold,
        'distance_to_stop': distance_to_stop,
        'weight': target_weights['weight']
    }).sort_values('distance_to_stop')
    
    print("\n📊 Stop-Loss Monitoring:")
    display(stop_loss_df.style
            .format({'current_drawdown': '{:.2%}', 'stop_loss_threshold': '{:.2%}', 
                    'distance_to_stop': '{:.2%}', 'weight': '{:.2%}'})
            .background_gradient(cmap='RdYlGn', subset=['distance_to_stop'], vmin=-0.05, vmax=0.1))
    
    # Check for positions near stop-loss
    at_risk = stop_loss_df[stop_loss_df['distance_to_stop'] < 0.02]  # Within 2% of stop
    if len(at_risk) > 0:
        print(f"\n🚨 ALERT: {len(at_risk)} position(s) near stop-loss threshold:")
        for ticker in at_risk.index:
            print(f"  {ticker}: {stop_loss_df.loc[ticker, 'current_drawdown']:.2%} drawdown " 
                  f"(threshold: {-stop_loss_threshold:.2%})")
    else:
        print("\n✅ All positions healthy - no stop-loss concerns")
    
    # Visualization: Distance to stop-loss
    fig = go.Figure()
    
    colors = ['red' if d < 0.02 else 'orange' if d < 0.05 else 'green' 
              for d in stop_loss_df['distance_to_stop']]
    
    fig.add_trace(go.Bar(
        x=stop_loss_df.index,
        y=stop_loss_df['distance_to_stop'] * 100,
        marker_color=colors,
        name='Distance to Stop-Loss'
    ))
    
    fig.add_hline(y=0, line_dash="dash", line_color="red", 
                  annotation_text="Stop-Loss Trigger")
    fig.add_hline(y=2, line_dash="dot", line_color="orange", 
                  annotation_text="Warning Zone (2%)")
    
    fig.update_layout(
        title=f'Distance to Stop-Loss Threshold ({vix_regime}: {stop_loss_threshold:.0%})',
        xaxis_title='ETF Ticker',
        yaxis_title='Distance to Stop (%)',
        height=500
    )
    
    fig.show()
    
    # VIX history chart
    fig2 = go.Figure()
    
    fig2.add_trace(go.Scatter(
        x=vix_series.index[-252:],  # Last year
        y=vix_series.iloc[-252:],
        mode='lines',
        name='VIX',
        line=dict(color='blue', width=2)
    ))
    
    # Add regime threshold lines
    fig2.add_hline(y=15, line_dash="dash", line_color="green", 
                   annotation_text="Low Vol (15% stop)")
    fig2.add_hline(y=25, line_dash="dash", line_color="orange", 
                   annotation_text="High Vol (10% stop)")
    
    fig2.update_layout(
        title='VIX History (Last Year) with Stop-Loss Regime Thresholds',
        xaxis_title='Date',
        yaxis_title='VIX Level',
        height=400
    )
    
    fig2.show()
    
else:
    print("⚠️  VIX data not found - using fixed 12% stop-loss threshold")
    print("   To enable dynamic stop-loss, download VIX data to:")
    print(f"   {vix_file}")
    
    # Still calculate drawdowns with fixed threshold
    stop_loss_threshold = 0.12
    lookback_days = 60
    recent_prices = prices[target_weights.index].iloc[-lookback_days:]
    running_max = recent_prices.expanding().max()
    drawdowns = (recent_prices - running_max) / running_max
    current_drawdowns = drawdowns.iloc[-1]
    distance_to_stop = stop_loss_threshold + current_drawdowns
    
    stop_loss_df = pd.DataFrame({
        'current_drawdown': current_drawdowns,
        'distance_to_stop': distance_to_stop,
        'weight': target_weights['weight']
    }).sort_values('distance_to_stop')
    
    print(f"\n📊 Stop-Loss Monitoring (Fixed 12% threshold):")
    display(stop_loss_df.style
            .format({'current_drawdown': '{:.2%}', 'distance_to_stop': '{:.2%}', 'weight': '{:.2%}'})
            .background_gradient(cmap='RdYlGn', subset=['distance_to_stop']))

---

## 6. Performance Tracking

Calculate returns since last rebalance and year-to-date performance.

In [ ]:
if executed_file.exists():
    # Determine last rebalance date (for now, assume 1 week ago or portfolio creation date)
    # In production, this would be tracked in a rebalance history file
    
    # Extract date from filename
    filename = latest_target_file.stem
    date_str = filename.split('_')[-2] + filename.split('_')[-1]
    portfolio_date = datetime.strptime(date_str, '%Y%m%d%H%M%S').date()
    
    print(f"📅 Portfolio Creation Date: {portfolio_date}")
    print(f"📅 Today: {datetime.now().date()}")
    days_since_creation = (datetime.now().date() - portfolio_date).days
    print(f"📅 Days Since Creation: {days_since_creation}")
    
    # Find closest date in price data
    portfolio_date_dt = pd.Timestamp(portfolio_date)
    valid_dates = prices.index[prices.index >= portfolio_date_dt]
    
    if len(valid_dates) > 0:
        start_date = valid_dates[0]
        end_date = prices.index[-1]
        
        print(f"\n📊 Performance Period: {start_date.date()} to {end_date.date()}")
        
        # Get price changes for holdings
        period_prices = prices.loc[start_date:end_date, target_weights.index]
        
        if len(period_prices) > 1:
            # Calculate returns
            returns = period_prices.pct_change().fillna(0)
            
            # Portfolio returns (weighted)
            weights_array = target_weights['weight'].values
            portfolio_returns = (returns * weights_array).sum(axis=1)
            
            # Cumulative returns
            cumulative_returns = (1 + portfolio_returns).cumprod() - 1
            
            # Calculate metrics
            total_return = cumulative_returns.iloc[-1]
            daily_vol = portfolio_returns.std()
            annualized_vol = daily_vol * np.sqrt(252)
            sharpe = (portfolio_returns.mean() * 252) / annualized_vol if annualized_vol > 0 else 0
            
            print(f"\n📈 Performance Metrics:")
            print(f"  Total Return: {total_return:.2%}")
            print(f"  Annualized Volatility: {annualized_vol:.2%}")
            print(f"  Sharpe Ratio (estimated): {sharpe:.2f}")
            
            # Cumulative return chart
            fig = go.Figure()
            
            fig.add_trace(go.Scatter(
                x=cumulative_returns.index,
                y=cumulative_returns * 100,
                mode='lines',
                name='Portfolio',
                line=dict(color='darkgreen', width=2),
                fill='tozeroy'
            ))
            
            fig.update_layout(
                title='Cumulative Return Since Portfolio Creation',
                xaxis_title='Date',
                yaxis_title='Cumulative Return (%)',
                height=500,
                hovermode='x unified'
            )
            
            fig.show()
            
            # Daily returns distribution
            fig2 = go.Figure()
            
            fig2.add_trace(go.Histogram(
                x=portfolio_returns * 100,
                nbinsx=30,
                name='Daily Returns',
                marker_color='steelblue'
            ))
            
            fig2.update_layout(
                title='Distribution of Daily Returns',
                xaxis_title='Daily Return (%)',
                yaxis_title='Frequency',
                height=400
            )
            
            fig2.show()
            
            # Individual position performance
            position_returns = (period_prices.iloc[-1] / period_prices.iloc[0] - 1)
            position_perf = pd.DataFrame({
                'return': position_returns,
                'weight': target_weights['weight'],
                'contribution': position_returns * target_weights['weight']
            }).sort_values('contribution', ascending=False)
            
            print("\n📊 Position Performance Contribution:")
            display(position_perf.style
                    .format({'return': '{:.2%}', 'weight': '{:.2%}', 'contribution': '{:.2%}'})
                    .background_gradient(cmap='RdYlGn', subset=['return', 'contribution']))
            
            # Contribution chart
            fig3 = go.Figure()
            
            colors = ['green' if c > 0 else 'red' for c in position_perf['contribution']]
            
            fig3.add_trace(go.Bar(
                x=position_perf.index,
                y=position_perf['contribution'] * 100,
                marker_color=colors,
                name='Contribution to Return'
            ))
            
            fig3.update_layout(
                title='Position Contribution to Portfolio Return',
                xaxis_title='ETF Ticker',
                yaxis_title='Contribution (%)',
                height=500
            )
            
            fig3.show()
        else:
            print("⚠️  Insufficient price data for performance calculation")
    else:
        print("⚠️  No price data available after portfolio creation date")
else:
    print("📝 No executed portfolio - performance tracking will be available after first trade execution")

---

## 7. Transaction Costs Summary

Track year-to-date transaction costs and rebalancing frequency.

In [ ]:
# In production, this would load from a transaction history file
# For now, estimate based on portfolio creation

print("📊 Transaction Cost Tracking:")
print("\nℹ️  Transaction history tracking not yet implemented.")
print("   This section will display:")
print("   - Total transaction costs YTD")
print("   - Number of rebalances YTD")
print("   - Average cost per rebalance")
print("   - Cost as % of portfolio value")
print("\n   Expected costs (based on validation):")
print("   - MVO: ~$660/year for $1M portfolio (0.066%)")
print("   - ~12 rebalances over 5 years (2.4/year average)")
print("   - ~$55 per rebalance")

# Placeholder for future implementation
# transaction_history_file = portfolio_dir / 'transaction_history.csv'
# if transaction_history_file.exists():
#     tx_history = pd.read_csv(transaction_history_file, parse_dates=['date'])
#     ytd_start = datetime(datetime.now().year, 1, 1)
#     ytd_txs = tx_history[tx_history['date'] >= ytd_start]
#     # Calculate and display metrics

---

## 8. Action Items & Alerts

Summary of any actions required based on monitoring.

In [ ]:
print("🔔 ACTION ITEMS SUMMARY")
print("=" * 60)

action_count = 0

# Check 1: Rebalancing needed?
if drift_df is not None and max_drift > 0.05:
    action_count += 1
    print(f"\n{action_count}. 🚨 REBALANCE REQUIRED")
    print(f"   Maximum drift: {max_drift:.2%} (threshold: 5.00%)")
    print(f"   Action: Execute trades to align with target portfolio")
    print(f"   Estimated cost: $50-100")

# Check 2: Stop-loss concerns?
if 'at_risk' in locals() and len(at_risk) > 0:
    action_count += 1
    print(f"\n{action_count}. ⚠️  STOP-LOSS WARNING")
    print(f"   {len(at_risk)} position(s) within 2% of stop-loss threshold")
    for ticker in at_risk.index:
        print(f"   - {ticker}: {stop_loss_df.loc[ticker, 'current_drawdown']:.2%} drawdown")
    print(f"   Action: Monitor closely, prepare to exit if threshold breached")

# Check 3: Data freshness
latest_data_date = prices.index[-1].date()
days_old = (datetime.now().date() - latest_data_date).days
if days_old > 2:
    action_count += 1
    print(f"\n{action_count}. ℹ️  DATA UPDATE RECOMMENDED")
    print(f"   Latest data: {latest_data_date} ({days_old} days old)")
    print(f"   Action: Run data collection script to update prices")
    print(f"   Command: python scripts/collect_etf_universe.py")

if action_count == 0:
    print("\n✅ No action items - portfolio status is healthy")
    print("   - All positions within drift threshold")
    print("   - No stop-loss concerns")
    print("   - Data is current")
    print("\n📅 Next review: Next Monday")
else:
    print(f"\n{'='*60}")
    print(f"Total action items: {action_count}")

---

## 9. How to Update This Dashboard

This notebook is designed to be run weekly as part of your portfolio monitoring routine.

### Weekly Workflow (Every Monday Morning)

#### Step 1: Update Price Data (5-10 minutes)
```bash
cd /home/stuar/code/ETFTrader
source venv/bin/activate
python scripts/collect_etf_universe.py
```

#### Step 2: Run This Notebook (2-3 minutes)
- Open notebook: `jupyter notebook notebooks/05_portfolio_monitoring_dashboard.ipynb`
- Kernel → Restart & Run All
- Review all sections

#### Step 3: Review Action Items
- Check Section 8 for any required actions
- If rebalancing needed, run portfolio generation script
- If stop-loss triggered, prepare exit orders

#### Step 4: Execute Trades (if needed)
- If drift > 5%, execute recommended trades
- Update `results/live_portfolio/executed_portfolio.csv` with new positions
- Record transaction costs in transaction history (future feature)

### Monthly Review
- Review performance trends
- Check factor score evolution
- Verify VIX regime and stop-loss adjustments are appropriate
- Update notes on portfolio behavior

### Quarterly Deep Dive
- Re-run full validation: `python scripts/08_backtest_real_data_3periods.py`
- Review `notebooks/04_real_data_validation_results.ipynb`
- Assess if any parameter adjustments needed
- Update technical documentation if methodology changes

---

## Notes & Observations

Use this section to track notes and observations over time:

**Week of [DATE]:**
- Portfolio status:
- Key observations:
- Actions taken:

---

## Questions?

See the [Operations Manual](../OPERATIONS_MANUAL.md) for detailed guidance on:
- Troubleshooting common issues
- Parameter tuning
- System maintenance
- Performance optimization

---

**Dashboard Complete** ✅
